In [ ]:
## This notebook shows how to create a customized popup info panel (that appears when you hover on an equipment):
## the operational limits data is added to the popup, in addition to the default equipment's attributes.

In [ ]:
import pypowsybl.network as pn
from pypowsybl_jupyter import network_explorer, display_nad

In [ ]:
#network = pn.create_eurostag_tutorial_example1_network()
network = pn.load('./data/MicroGridTestConfiguration_T4_BE_BB_Complete_v2.zip', {'iidm.import.cgmes.post-processors': 'cgmesGLImport'})

In [ ]:
# create a function that returns the data as html, using the pandas' style APIs

def get_equipment_info_with__operational_limits(id, type):

    ## retrieve the equipment's operational limits as a dataframe
    def get_operational_limits_df(network, id):
        operational_limits=network.get_operational_limits(attributes=["side", "name", "type", "value", "acceptable_duration", "fictitious"])
        limits1=None
        if id in operational_limits.index:
            limits1 = operational_limits.loc[id]
        return limits1

    ## return the equipment's attributes, combined with its operational limits (if they exist), in HTML
    def format_to_html_table(row, support_df, id, type, type2):
        table_style =  [
                    {
                        "selector": "caption",
                        "props": "caption-side: top; font-weight: bold; background-color: #f8f8f8; border-bottom: 1px solid #ddd; width: fit-content; white-space: nowrap;",
                    },
                    {
                        "selector": "th",
                        "props": "text-align: left; font-weight: bold; background-color: #f8f8f8;",
                    },
                    {
                        "selector": "td",
                        "props": "text-align: left;",
                    },
                ]

        # the equipment's attributes
        table = (
            row.to_frame()
            .style.set_caption(f"{type}: {id}")
            .set_table_styles(table_style)
            .format(precision=3, thousands=".", decimal=",")
            .set_table_attributes('border="0"')
            .hide(axis="columns")
            .to_html()
        )

        if support_df is not None:
            html_template = """
                <div style='display: flex; gap: 20px;'>
                    <div style='border-collapse: collapse; font-size: 0.9em;'>{df1_html}</div>
                    <div style='border-collapse: collapse; font-size: 0.9em;'><br><br>{df2_html}</div>
                </div>
            """

            # the equipment's operational limits
            table2 = (
                support_df
                .style
                .set_caption(f"{type2}")
                .set_table_styles(table_style)
                .format({'value': lambda x: f"{x:.3e}" if abs(x) > 1e6 else f"{x:.3f}" })
                .hide(axis="rows")
                .to_html()
            )
            return  html_template.format(df1_html=table, df2_html=table2)
        else:
            return table

    # process the equipment by its type
    if type == 'LINE':
        line  = network.get_lines().loc[id]
        limits_df = get_operational_limits_df(network, id)
        return format_to_html_table(line, limits_df, id, type, 'Operational limits')
    elif type == 'HVDC_LINE':
        return format_to_html_table(network.get_hvdc_lines().loc[id], None,  id, type, None)
    elif type in [ 'PHASE_SHIFT_TRANSFORMER', 'TWO_WINDINGS_TRANSFORMER']:
        twt=network.get_2_windings_transformers().loc[id]
        limits_df = get_operational_limits_df(network, id)
        return format_to_html_table(twt, limits_df, id, type, 'Operational limits')
    elif type == 'THREE_WINDINGS_TRANSFORMER':
        three_wt = network.get_3_windings_transformers().loc[id]
        limits_df = get_operational_limits_df(network, id)
        return format_to_html_table(three_wt, limits_df, id, type, 'Operational limits')
    elif type == 'DANGLING_LINE':
        line  = network.get_dangling_lines().loc[id]
        limits_df = get_operational_limits_df(network, id)
        return format_to_html_table(line, limits_df, id, type, 'Operational limits')
    return ""

In [ ]:
network_explorer(network, on_hover_func=get_equipment_info_with__operational_limits)